# Model pipeline draft

In [137]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import layers, models, Sequential, Input, Model
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.data.experimental import cardinality
from tensorflow.data.experimental import AUTOTUNE
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

In [74]:
path0 = r'C:\Users\pitip\code\ClaireLeroux44\ArtRecognition'
n_artist = 3

Steps:
- load images from dataset (after folder prep) : image from dataset
- load VGG16
- change VGG16 last layer (add N artist id)
- train VGG16 on dataset
- predict artist on test set + accuracy

In [75]:
def VGG16_model(ndense_1=100):
    base_model = VGG16(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
    base_model.trainable = False

    model_layers = [base_model, layers.Flatten(), 
                  layers.Dense(ndense_1, activation='relu')]
    model = models.Sequential(layers = model_layers)
    return model

model1 = VGG16_model(ndense_1=100)
model1.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_3 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_10 (Dense)             (None, 100)               2508900   
Total params: 17,223,588
Trainable params: 2,508,900
Non-trainable params: 14,714,688
_________________________________________________________________


In [123]:
train_dir = os.path.join(path0, 'raw_data', 'test_VGG16', 'Train')
test_dir = os.path.join(path0, 'raw_data', 'test_VGG16', 'Test')
BATCH_SIZE = 32
IMG_SIZE = (224, 224)

train_dataset = image_dataset_from_directory(train_dir, shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE, label_mode='categorical')
test_dataset = image_dataset_from_directory(test_dir, shuffle=True, batch_size=BATCH_SIZE, image_size=IMG_SIZE, label_mode='categorical')

Found 15 files belonging to 3 classes.
Found 9 files belonging to 3 classes.


In [124]:
class_names = train_dataset.class_names
class_names

['class_1', 'class_2', 'class_3']

In [125]:
train_batches = cardinality(train_dataset)
validation_dataset = train_dataset.take(train_batches // 5)
train_train_dataset = train_dataset.skip(train_batches // 5)

In [126]:
train_train_dataset = train_train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)
test_dataset = test_dataset.prefetch(buffer_size=AUTOTUNE)

In [127]:
for image_batch, labels_batch in train_train_dataset:
    print(image_batch.shape)
    print(labels_batch.shape)

(15, 224, 224, 3)
(15, 3)


In [ ]:
datagen = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    #rotation_range=10,
    width_shift_range=0.2,
    height_shift_range=0.2,
#    horizontal_flip=True,
    zoom_range=(0.8, 1.2),) 
datagen.fit(train_dataset)

In [141]:
data_augmentation = Sequential([layers.experimental.preprocessing.RandomFlip('horizontal'),
                                layers.experimental.preprocessing.RandomRotation(0.1),
                                layers.experimental.preprocessing.RandomContrast(0.1),
                                layers.experimental.preprocessing.RandomZoom(height_factor=0.2),
                                layers.experimental.preprocessing.RandomTranslation(height_factor=0.2, width_factor=0.2)
])

In [142]:
#image_batch, label_batch = next(iter(train_dataset))
#feature_batch = base_model(image_batch)
#print(feature_batch.shape)

In [143]:
inputs = Input(shape=(224, 224, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = model1(x)
prediction_layer = layers.Dense(n_artist, activation='softmax')
outputs = prediction_layer(x)
model = Model(inputs, outputs)

In [144]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics='accuracy')

In [145]:
model.summary()

Model: "functional_18"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_14 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
sequential_9 (Sequential)    (None, 224, 224, 3)       0         
_________________________________________________________________
tf_op_layer_strided_slice_9  [(None, 224, 224, 3)]     0         
_________________________________________________________________
tf_op_layer_BiasAdd_9 (Tenso [(None, 224, 224, 3)]     0         
_________________________________________________________________
sequential_5 (Sequential)    (None, 100)               17223588  
_________________________________________________________________
dense_15 (Dense)             (None, 3)                 303       
Total params: 17,223,891
Trainable params: 2,509,203
Non-trainable params: 14,714,688
_________________________________

In [146]:
history = model.fit(train_train_dataset, epochs=10, validation_data=test_dataset, batch_size=32, 
                    callbacks=[EarlyStopping(patience=2, restore_best_weights=True)])

Epoch 1/10
1/1 [==============================] - 2s 2s/step - loss: 38.4901 - accuracy: 0.3333 - val_loss: 56.5130 - val_accuracy: 0.5556
Epoch 2/10
1/1 [==============================] - 1s 1s/step - loss: 18.1203 - accuracy: 0.6667 - val_loss: 70.6977 - val_accuracy: 0.4444
Epoch 3/10
1/1 [==============================] - 2s 2s/step - loss: 11.6085 - accuracy: 0.6667 - val_loss: 78.9916 - val_accuracy: 0.4444
